# Test cases reference for histogram equalization

In [1]:
import numpy as np


from bokeh.plotting import figure, show, output_notebook
from bokeh.palettes import mpl
from bokeh.models import LinearColorMapper, LogColorMapper, EqHistColorMapper

In [2]:
output_notebook()

Loading BokehJS ...

Datashader's `eq_hist` implementation (originally from scikit image):

In [3]:
def eq_hist(data, mask=None, nbins=256*256):
    if not isinstance(data, np.ndarray):
        raise TypeError("data must be np.ndarray")
    data2 = data if mask is None else data[~mask]
    if data2.dtype == bool or np.issubdtype(data2.dtype, np.integer):
        hist = np.bincount(data2.ravel())
        bin_centers = np.arange(len(hist))
        idx = np.nonzero(hist)[0][0]
        hist, bin_centers = hist[idx:], bin_centers[idx:]
    else:
        hist, bin_edges = np.histogram(data2, bins=nbins)
        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    cdf = hist.cumsum()
    cdf = cdf / float(cdf[-1])
    out = np.interp(data.flat, bin_centers, cdf).reshape(data.shape)
    return out if mask is None else np.where(mask, np.nan, out)


Helper methods to compare eq_hist implementations or compare with other colormappers:

In [4]:
def show_eq_hist(d, bokeh=True):
    "If bokeh=True, use client side colormapping, otherwise display data with eq_hist applied"
    
    mapper = EqHistColorMapper if bokeh else LinearColorMapper
    if not bokeh:
        d = eq_hist(d)
    color_mapper = mapper(palette=mpl['Plasma'][256], low=d.min(), high=d.max())
    p = figure(x_range=(0, 10), y_range=(0, 10), plot_width=300, plot_height=300,
               tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")])
    p.image(image=[d], x=0, y=0, dw=10, dh=10, color_mapper=color_mapper)
    show(p)
    
def show_comparison(d, mapper):
    "Useful for comparison" 
    color_mapper = LinearColorMapper(palette=mpl['Plasma'][256], low=d.min(), high=d.max())
    p = figure(x_range=(0, 10), y_range=(0, 10), plot_width=300, plot_height=300,
               tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")])
    p.image(image=[d], x=0, y=0, dw=10, dh=10, color_mapper=color_mapper)
    show(p)

Functions to generate example data:

In [5]:
def pipeline_example(num=10000):
    return np.load('pipeline.npy').astype(float)

def sparse_squares():
    d = np.zeros((100,100))
    d[5:10,5:10] = 10
    d[50:60,50:60] = 100
    d[70:80,70:80] = 1000
    return d

def simple_noise():
    np.random.seed(42)
    return (np.random.uniform(size=(10,10))**10)

def simple_gaussian():
    x, y = np.meshgrid(np.linspace(-5, 5), np.linspace(-5, 5))
    return 1. / (2. * np.pi) * np.exp(-(x**2. / 2.  + y**2. / 2.))

def sparse_cells():
    N = 100
    x = np.linspace(0, 10, N)
    y = np.linspace(0, 10, N)
    xx, yy = np.meshgrid(x, y)
    d = np.sin(xx)*np.cos(yy)
    d[5,5] = 10
    return d

## Datashader pipeline example

In [6]:
pipeline = pipeline_example()

### Bokeh colormapping

In [7]:
show_eq_hist(pipeline, bokeh=True)

### Datashader Reference 

In [8]:
show_eq_hist(pipeline, bokeh=False)

### Linear reference

Only showing the linear case for this test case

In [9]:
show_comparison(pipeline, mapper=LinearColorMapper)

## Simple sparse squares example

In [10]:
squares = sparse_squares()

### Bokeh colormapping

In [11]:
show_eq_hist(squares, bokeh=True)

### Datashader Reference 

In [12]:
show_eq_hist(squares, bokeh=False)

## Simple noise example raised to power 10

In [13]:
noise = simple_noise()

### Bokeh colormapping

In [14]:
show_eq_hist(noise, bokeh=True)

### Datashader Reference 

In [15]:
show_eq_hist(noise, bokeh=False)

## Simple gaussian example

In [16]:
gaussian = simple_gaussian()

### Bokeh colormapping

In [17]:
show_eq_hist(gaussian, bokeh=True)

### Datashader Reference 

In [18]:
show_eq_hist(gaussian, bokeh=False)

## Sparse cells example

In [19]:
cells = sparse_cells()

### Bokeh colormapping

In [20]:
show_eq_hist(cells, bokeh=True)

### Datashader Reference 

In [21]:
show_eq_hist(cells, bokeh=False)